# 대화 속 Speaker의 utterance 개수 분포

- 데이터를 보니 ("nikluge-2024-일상 대화 요약-test-000178")와 같은 샘플은 speaker1이 한 문장만 말하고, 나머지는 모두 speaker2가 말하는 경우가 있음
    - 위와 같은 경우가 얼마나 많이 존재하는지
    - 이 경우에는 output의 구문 구조가 어떻게 변하는지를 확인

In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from transformers import AutoTokenizer

In [3]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [28]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')

In [29]:
def get_num_utterances_per_speaker(df: pd.DataFrame):
    """
    Get the number of utterances per speaker in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the data.
    """
    df = deepcopy(df)

    # Get the number of utterances per speaker
    def return_num_utterances_per_speaker(turns):
        speakers = [turn['speaker'] for turn in turns]
        if speakers is None:
            print("errror")
        return pd.Series(speakers).value_counts().to_dict()
    
    df['num_utterances_per_speaker'] = df['conversation'].apply(return_num_utterances_per_speaker)
    
    return df

In [30]:
train_df = get_num_utterances_per_speaker(train_df)
dev_df = get_num_utterances_per_speaker(dev_df)
test_df = get_num_utterances_per_speaker(test_df)

In [33]:
train_df.head(1)

,id,conversation,subject_keyword,speakers,output,num_utterances_per_speaker
0,nikluge-2024-일상 대화 요약-train-000001,"[{'speaker': 'SD2000001', 'utterance': '저는 여행 ...",[해외여행],"[SD2000001, SD2000002]",이 대화에서 화자들은 좋았던 여행지와 기억나는 주요 명소에 대해 이야기했습니다. S...,"{'SD2000002': 8, 'SD2000001': 6}"


<br/>

### speaker1 또는 speaker2의 utterance가 conversation 내에 단 1개만 존재하는 샘플 추출

- train : 3개, dev : 0개, test : 9개

In [38]:
def extract_one_utterance_samples(df: pd.DataFrame):
    """
    Extract samples with only one utterance in the conversation.

    Parameters:
    df (pd.DataFrame): DataFrame of the data.

    Returns:
    pd.DataFrame: DataFrame with only one utterance in the conversation.
    """
    df = deepcopy(df)

    # Extract samples with only one utterance in the conversation
    def is_one_utterance(row):
        is_one = [True if num == 1 else False for num in row['num_utterances_per_speaker'].values()]
        return any(is_one)

    special_samples = df.apply(is_one_utterance, axis=1)

    return special_samples

In [39]:
train_df[extract_one_utterance_samples(train_df)]

,id,conversation,subject_keyword,speakers,output,num_utterances_per_speaker
19,nikluge-2024-일상 대화 요약-train-000020,"[{'speaker': 'SD2000038', 'utterance': '전원주택을 ...","[전원주택, 재테크]","[SD2000039, SD2000038]",두 화자는 이 대화에서 SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에...,"{'SD2000039': 4, 'SD2000038': 1}"
87,nikluge-2024-일상 대화 요약-train-000088,"[{'speaker': 'SD2000968', 'utterance': '그리고 요즘...","[에이아이 면접, 화상 면접]","[SD2000966, SD2000968]",두 화자는 이 대화에서 AI 면접에 대해 이야기했습니다. SD2000966은 상대방...,"{'SD2000968': 13, 'SD2000966': 1}"
176,nikluge-2024-일상 대화 요약-train-000177,"[{'speaker': 'SD2001336', 'utterance': '혹시 nam...",[유기견],"[SD2001335, SD2001336]",두 화자는 이 대화에서 유기견에 대해 말했습니다. SD2001336은 상대방에게 유...,"{'SD2001335': 6, 'SD2001336': 1}"


In [40]:
dev_df[extract_one_utterance_samples(dev_df)]

,id,conversation,subject_keyword,speakers,output,num_utterances_per_speaker


In [42]:
test_df[extract_one_utterance_samples(test_df)]

,id,conversation,subject_keyword,speakers,output,num_utterances_per_speaker
18,nikluge-2024-일상 대화 요약-test-000019,"[{'speaker': 'SD2000060', 'utterance': '그럼 혹시 ...","[반려동물, 경험]","[SD2000060, SD2000061]",,"{'SD2000061': 8, 'SD2000060': 1}"
149,nikluge-2024-일상 대화 요약-test-000150,"[{'speaker': 'SD2002378', 'utterance': '약간 남자의...",[첫사랑],"[SD2002377, SD2002378]",,"{'SD2002377': 9, 'SD2002378': 1}"
153,nikluge-2024-일상 대화 요약-test-000154,"[{'speaker': 'SD2000190', 'utterance': '근데 요즘 ...",[음식],"[SD2000191, SD2000190]",,"{'SD2000191': 16, 'SD2000190': 1}"
163,nikluge-2024-일상 대화 요약-test-000164,"[{'speaker': 'SD2000205', 'utterance': '선생님은 어...",[운동],"[SD2000205, SD2000204]",,"{'SD2000204': 14, 'SD2000205': 1}"
177,nikluge-2024-일상 대화 요약-test-000178,"[{'speaker': 'SD2000216', 'utterance': 'name1 ...",[알바],"[SD2000217, SD2000216]",,"{'SD2000217': 7, 'SD2000216': 1}"
189,nikluge-2024-일상 대화 요약-test-000190,"[{'speaker': 'SD2000240', 'utterance': '아르바이트 ...","[아르바이트, 기억]","[SD2000239, SD2000240]",,"{'SD2000239': 9, 'SD2000240': 1}"
225,nikluge-2024-일상 대화 요약-test-000226,"[{'speaker': 'SD2000298', 'utterance': '사모님께서는...",[관리],"[SD2000299, SD2000298]",,"{'SD2000299': 4, 'SD2000298': 1}"
307,nikluge-2024-일상 대화 요약-test-000308,"[{'speaker': 'SD2002405', 'utterance': '선생님은 만...",[도전],"[SD2002406, SD2002405]",,"{'SD2002406': 14, 'SD2002405': 1}"
354,nikluge-2024-일상 대화 요약-test-000355,"[{'speaker': 'SD2000551', 'utterance': '선생님은 인...","[여행, 여행]","[SD2000550, SD2000551]",,"{'SD2000550': 3, 'SD2000551': 1}"


<br/>

### 이런 샘플의 경우 output의 구문 구조가 바뀌는가?

- **아니다.**

    *["nikluge-2024-일상 대화 요약-train-000020"]*

    "output": "두 화자는 이 대화에서 SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다. SD2000038은 상대방이 전원주택에 살기 위해 어떤 준비를 하고 있는지 물어봤습니다. SD2000039는 재테크에는 관심이 없었지만 전원주택에서 살기 위한 준비로 아르바이트도 하며 돈을 열심히 모으고 있다고 말했습니다."

    <br/>

    *["nikluge-2024-일상 대화 요약-train-000088"]*
    
    "output": "두 화자는 이 대화에서 AI 면접에 대해 이야기했습니다. SD2000966은 상대방에게 AI 면접과 화상 면접이 어떤 차이가 있는지 물었습니다. SD2000968은 최근 AI 면접을 많이 진행하고 있는 추세이고 교육 분야 역시 코로나로 인해 이런 변화들이 빠르게 나타나고 있다고 말했습니다. 또 화상 면접은 랜선을 통해 실시간으로 면접을 진행하지만 AI 면접은 애플리케이션에 접속해서 촬영한 영상만 업로드하면 되기 때문에 회사와 개인 간의 비용과 시간을 절감할 수 있는 장점이 있다고 말했습니다."

    <br/>
    
    *["nikluge-2024-일상 대화 요약-train-000177"]*

    "output": "두 화자는 이 대화에서 유기견에 대해 말했습니다. SD2001336은 상대방에게 유기견에 대한 견해를 물어보았습니다. SD2001335는 유기견에 대해 깊게 생각해 본 적은 없지만, 상황이 허락한다면 기를 마음이 있다고 말했습니다. 또한 가족과 같은 애완동물을 버리는 사람은 도덕적으로 결함이 있다며 비난했습니다. 그리고 고양이든 강아지든 한번 키웠으면 끝까지 지켜주고 의무를 다해야 한다고 말했습니다."

<br/>

- train 내 3개의 샘플 모두 아래와 같은 구조로 답해야함

    - `전반적인 요약` : "두 화자는 이 대화에서 ~에 대해 말했습니다" 
    - `speaker 1 요약` 
    - `speaker 2 요약`
        
<br/>

### 실제 학습된 모델의 경우 9개의 test 샘플에 대해 어떻게 답변하는가?

- hyperstellar

    - `000019` : {'SD2000061': 8, 'SD2000060': 1}\
        'SD2000061' 에 대해서만 요약

    - `000150` : {'SD2002377': 9, 'SD2002378': 1}\
        'SD2002377' 에 대해서만 요약

    - `000154` : {'SD2000191': 16, 'SD2000190': 1}\
        'SD2000190'와 'SD2000191'둘 다 요약이 포함되었지만, 전반적인 요약은 없음

    - `000164` : {'SD2000204': 14, 'SD2000205': 1}\
        'SD2000204'에 대해서만 요약

    - `000178` : {'SD2000217': 7, 'SD2000216': 1}\
        'SD2000217'에 대해서만 요약

    - `000190` : {'SD2000239': 9, 'SD2000240': 1}\
        'SD2000239'에 대해서만 요약

    - `000226` : {'SD2000299': 4, 'SD2000298': 1}\
        'SD2000298'와 'SD2000299'둘 다 요약이 포함되었지만, 전반적인 요약은 없음

    - `000308` : {'SD2002406': 14, 'SD2002405': 1}\
        'SD2002406'에 대해서만 요약

    - `000355` : {'SD2000550': 3, 'SD2000551': 1}\
        'SD2000551'와 'SD2000550'둘 다 요약이 포함되었지만, 전반적인 요약은 없음

<br/>

- 내가내가내가내가내가

    - `000019` : {'SD2000061': 8, 'SD2000060': 1}\
        3가지 요약 모두 포함

    - `000150` : {'SD2002377': 9, 'SD2002378': 1}\
        전반적인 요약 + 'SD2002377' 에 대해서만 요약 / 근데 화자를 틀림 'SD 203777'

    - `000154` : {'SD2000191': 16, 'SD2000190': 1}\
        3가지 요약 모두 포함. 그런데, 화자를 틀림 'SD 2000 190', 'SD 290191'

    - `000164` : {'SD2000204': 14, 'SD2000205': 1}\
        3가지 요약 모두 포함. 그런데, 화자를 틀림 

    - `000178` : {'SD2000217': 7, 'SD2000216': 1}\
        3가지 요약 모두 포함. 그런데, 화자를 틀림 

    - `000190` : {'SD2000239': 9, 'SD2000240': 1}\
        3가지 요약 모두 포함. 그런데, 화자를 틀림 < 조금 특이하게, 화자를 "SD~~"가 아닌 "화자"라고 함 >

    - `000226` : {'SD2000299': 4, 'SD2000298': 1}\
        전반적인 요약만 존재

    - `000308` : {'SD2002406': 14, 'SD2002405': 1}\
        3가지 요약 모두 포함

    - `000355` : {'SD2000550': 3, 'SD2000551': 1}\
        3가지 요약 모두 포함, 그런데 화자를 틀림